# Limpeza e tratamento dos dados da [CEAPS](https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps?utm_source=ActiveCampaign&utm_medium=email&utm_content=%237DaysOfCode+-+Ci%C3%AAncia+de+Dados+1%2F7%3A+Data+Cleaning+and+Preparation&utm_campaign=%5BAlura+%237Days+Of+Code%5D%28Java%29+Dia+1%2F7%3A+Consumir+uma+API+de+filmes)


### Importa as bibliotecas utilizadas

In [195]:
import pandas as pd
import re

### Cria as funções utilizadas no tratamento dos arquivos

In [196]:
def import_and_treatment_and_agregate(csvs):
    """
    Esta função realiza a leitura dos arquivos csv informados.

    Parâmetros:
    csvs (str): Lista de strings com os nomes de cada arquivo.

    Retorna:
    dataframe: Retorna a junção de todas as base de dados no formato de dataframe do Pandas.
    """
    df = pd.DataFrame()
    for csv in csvs:
        df_aux = pd.read_csv(f"Despesas CEAPS/{csv}", encoding='ISO-8859-1',delimiter=';',skiprows=1)
        df = df.append(df_aux,ignore_index=True)
    return df

def exclui_coluna(df):
    """
    Esta função realiza a exclusão da coluna 'DETALHAMENTO'.

    Parâmetros:
    df (Dataframe): O Dataframe que sofrerá a alteração.

    Retorna:
    df: O Dataframe após a exclusão da coluna.
    """
    df.drop('DETALHAMENTO',axis=1,inplace=True)
    return df

def renomeia_coluna(df):
    """
    Esta função realiza a troca dos nomes das colunas dos dataframes.

    Parâmetros:
    df (Dataframe): O Dataframe que sofrerá a alteração.

    Retorna:
    df: O Dataframe após a alteração dos nomes das colunas.
    """
    df.rename(columns={
        "ANO":"Ano",
        "MES":"Mes",
        "SENADOR":"Senador",
        "TIPO_DESPESA":"Tipo Desp.",
        "CNPJ_CPF":"CNPJ/CPF",
        "FORNECEDOR":"Fornecedor",
        "DOCUMENTO":"Documento",
        "DATA":"Data",
        "VALOR_REEMBOLSADO":"Reembolso",
        "COD_DOCUMENTO":"Cod. Documento"
    },inplace=True)
    return df

def verifica_formato_data(row):
    """
    Esta função verifica se o formato da data esta ok e corrige caso contrario.

    Parâmetros:
    row (Series): O Dataframe que sofrerá a alteração.

    Retorna:
    (str): A string com o formato correto de data.
    """
    regex_data = r'^(0?[1-9]|[12]\d|3[01])/(0?[1-9]|1[0-2])/((?:19|20)\d{2})$'
    if re.match(regex_data,row['Data']):
        return row['Data']
    else:
        dia,mes,ano = row['Data'].split('/')
        ano = ano[1]+ano[0]+ano[2]+ano[3]
        if int(ano) < 2008:
            ano = ano[0]+ano[1]+ano[3]+ano[2]
        return dia+'/'+mes+'/'+ano

def converte_as_colunas(df):
    """
    Esta função realiza a conversão dos tipos das colunas.

    Parâmetros:
    df (Dataframe): O Dataframe que sofrerá a alteração.

    Retorna:
    df: O Dataframe após a alteração dos nomes das colunas.
    """
    df['Data'] = df.apply(verifica_formato_data, axis=1)
    df['Data'] = pd.to_datetime(df['Data'],format="%d/%m/%Y")
    df['Reembolso'] = df['Reembolso'].str.replace(',', '.')
    df['Senador'] = df['Senador'].str.replace('É','E')
    df['Senador'] = df['Senador'].str.replace('Ô','O')
    df['Senador'] = df['Senador'].str.replace('Ê','E')
    df['Reembolso'] = df['Reembolso'].astype(float).round(2)
    df['Cod. Documento'] = df['Cod. Documento'].astype(str)
    
    return df

### Define quais arquivos serão lidos

In [197]:
csvs = ['despesa_ceaps_2022.csv',
        'despesa_ceaps_2021.csv',
        'despesa_ceaps_2020.csv',
        'despesa_ceaps_2019.csv']

### Executo as operações definidas anteriormente

In [198]:
df = import_and_treatment_and_agregate(csvs)
df = exclui_coluna(df)
df = renomeia_coluna(df)
df = converte_as_colunas(df)
df

,Ano,Mes,Senador,Tipo Desp.,CNPJ/CPF,Fornecedor,Documento,Data,Reembolso,Cod. Documento
0,2022,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/22,2022-01-03,6000.00,2173614
1,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,26.320.603/0001-64,INFORMANAHORA,000000000000310/A,2022-01-04,1500.00,2173615
2,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,13.659.201/0001-47,LINHA PURPURA FOTO E VIDEO LTDA,107,2022-01-14,6000.00,2173616
3,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,23.652.846/0001-01,ROBERTO GUTIERREZ DA ROCHA M.E.I.,187,2022-01-18,1000.00,2173618
4,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,08.941.827/0001-01,RONDONIA DINÂMICA COM. E SERV. DE INFORMÁTICA ...,000000000001772/A,2022-01-17,2000.00,2173617
...,...,...,...,...,...,...,...,...,...,...
69328,2019,1,ZEZE PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",09.296.295/0001-60,Azul,O954UL,2019-01-19,769.29,2113825
69329,2019,1,ZEZE PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",09.296.295/0001-60,Azul,G9Z36V,2019-01-24,310.62,2113828
69330,2019,1,ZEZE PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",07.575.651/0001-59,GOL,WNQI4A,2019-01-29,1168.10,2114101
69331,2019,1,ZEZE PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",33.937.681/0001-78,LATAM,NCCPIM,2019-01-23,772.05,2113826


In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69333 entries, 0 to 69332
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Ano             69333 non-null  int64         
 1   Mes             69333 non-null  int64         
 2   Senador         69333 non-null  object        
 3   Tipo Desp.      69333 non-null  object        
 4   CNPJ/CPF        69333 non-null  object        
 5   Fornecedor      69333 non-null  object        
 6   Documento       66370 non-null  object        
 7   Data            69333 non-null  datetime64[ns]
 8   Reembolso       69333 non-null  float64       
 9   Cod. Documento  69333 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 5.3+ MB


In [200]:
df_legislação_56 = pd.read_csv('56.csv',encoding='utf-8',delimiter=';')
columns = df_legislação_56.columns
new_columns =[]
for column in columns:
    column = column.replace('/Servico/Parlamentares/Parlamentar/IdentificacaoParlamentar/','')
    column = column.replace('/Servico/Parlamentares/Parlamentar/Mandatos/Mandato/','-')
    new_columns.append(column)
df_legislação_56.columns = new_columns
filtro = 'Exercicios'
drops = [s for s in df_legislação_56.columns if filtro in s]
df_legislação_56.drop(columns=drops, inplace=True)
filtro = 'Suplentes'
drops = [s for s in df_legislação_56.columns if filtro in s]
df_legislação_56.drop(columns=drops, inplace=True)
filtro = 'PrimeiraLegislaturaDoMandato'
drops = [s for s in df_legislação_56.columns if filtro in s]
df_legislação_56.drop(columns=drops, inplace=True)
filtro = 'SegundaLegislaturaDoMandato'
drops = [s for s in df_legislação_56.columns if filtro in s]
df_legislação_56.drop(columns=drops, inplace=True)
filtro = 'Titular'
drops = [s for s in df_legislação_56.columns if filtro in s]
df_legislação_56.drop(columns=drops, inplace=True)
df_legislação_56.drop(columns=['-CodigoMandato','-DescricaoParticipacao'], inplace=True)
df_legislação_56.drop(df_legislação_56.columns[-1], axis=1, inplace=True)
df_legislação_56.drop_duplicates(inplace=True)
mascara = df_legislação_56['-UfParlamentar'].isna()
df_legislação_56 = df_legislação_56.drop(df_legislação_56[mascara].index)
df_legislação_56.rename(columns={
        "NomeParlamentar":"Senador"
    },inplace=True)
df_legislação_56['Senador'] =df_legislação_56['Senador'].str.upper()


df_legislação_55 = pd.read_csv('55.csv',encoding='utf-8',delimiter=';')
columns = df_legislação_55.columns
new_columns =[]
for column in columns:
    column = column.replace('/Servico/Parlamentares/Parlamentar/IdentificacaoParlamentar/','')
    column = column.replace('/Servico/Parlamentares/Parlamentar/Mandatos/Mandato/','-')
    new_columns.append(column)
df_legislação_55.columns = new_columns
filtro = 'Exercicios'
drops = [s for s in df_legislação_55.columns if filtro in s]
df_legislação_55.drop(columns=drops, inplace=True)
filtro = 'Suplentes'
drops = [s for s in df_legislação_55.columns if filtro in s]
df_legislação_55.drop(columns=drops, inplace=True)
filtro = 'PrimeiraLegislaturaDoMandato'
drops = [s for s in df_legislação_55.columns if filtro in s]
df_legislação_55.drop(columns=drops, inplace=True)
filtro = 'SegundaLegislaturaDoMandato'
drops = [s for s in df_legislação_55.columns if filtro in s]
df_legislação_55.drop(columns=drops, inplace=True)
filtro = 'Titular'
drops = [s for s in df_legislação_55.columns if filtro in s]
df_legislação_55.drop(columns=drops, inplace=True)
df_legislação_55.drop(columns=['-CodigoMandato','-DescricaoParticipacao'], inplace=True)
df_legislação_55.drop(df_legislação_55.columns[-1], axis=1, inplace=True)
df_legislação_55.drop_duplicates(inplace=True)
mascara = df_legislação_55['-UfParlamentar'].isna()
df_legislação_55 = df_legislação_55.drop(df_legislação_55[mascara].index)
df_legislação_55.rename(columns={
        "NomeParlamentar":"Senador"
    },inplace=True)
df_legislação_55['Senador'] = df_legislação_55['Senador'].str.upper()

df_legislação_56['Senador'] = df_legislação_56['Senador'].str.replace('É','E')
df_legislação_56['Senador'] = df_legislação_56['Senador'].str.replace('Ô','O')
df_legislação_56['Senador'] = df_legislação_56['Senador'].str.replace('Ê','E')
mapeamento = {'MARIA ELIZA': 'MARIA ELIZA DE AGUIAR E SILVA',
            'JEAN-PAUL PRATES': 'JEAN PAUL PRATES',
            'DR. SAMUEL ARAÚJO': 'SAMUEL ARAUJO',
            'WEVERTON': 'WEVERTON ROCHA',
            'PEDRO CHAVES': 'PEDRO PINHEIRO CHAVES'}

df_legislação_56['Senador'] = df_legislação_56['Senador'].replace(mapeamento)
dados = [{'CodigoParlamentar':5070,
            'Senador':'WILDER MORAIS',
            'NomeCompletoParlamentar':'Wilder Pedro de Morais',
            'SexoParlamentar':'Masculino',
            'FormaTratamento':'Senador',
            'SiglaPartidoParlamentar':'PL',
            '-UfParlamentar':'GO',
            'CodigoPublicoNaLegAtual':870,
            'UrlFotoParlamentar':'http://www.senado.leg.br/senadores/img/fotos-oficiais/senador5070.jpg',
            'UrlPaginaParlamentar':'',
            'UrlPaginaParticular':'',
            'EmailParlamentar':'a'
}]
nova_linha = pd.DataFrame(dados)
df_legislação_56 = df_legislação_56.append(nova_linha, ignore_index=True)

df_legislação_55['Senador'] = df_legislação_55['Senador'].str.replace('É','E')
df_legislação_55['Senador'] = df_legislação_55['Senador'].str.replace('Ô','O')
df_legislação_55['Senador'] = df_legislação_55['Senador'].str.replace('Ê','E')
mapeamento = {'MARIA ELIZA': 'MARIA ELIZA DE AGUIAR E SILVA',
            'JEAN-PAUL PRATES': 'JEAN PAUL PRATES',
            'DR. SAMUEL ARAÚJO': 'SAMUEL ARAUJO',
            'WEVERTON': 'WEVERTON ROCHA',
            'PEDRO CHAVES': 'PEDRO CHAVES DOS SANTOS FILHO'}

df_legislação_55['Senador'] = df_legislação_55['Senador'].replace(mapeamento)

df_legislação = df_legislação_55.append(df_legislação_56,ignore_index=True)
df_legislação.drop_duplicates(inplace=True)
df_legislação.drop(columns=['FormaTratamento',
                            'CodigoParlamentar',
                            'UrlPaginaParlamentar',
                            'UrlPaginaParticular',
                            'EmailParlamentar',
                            'CodigoPublicoNaLegAtual'], inplace=True)
df_completo = pd.merge(df, df_legislação, on='Senador', how='left')

display(df.info())
display(df_completo.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69333 entries, 0 to 69332
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Ano             69333 non-null  int64         
 1   Mes             69333 non-null  int64         
 2   Senador         69333 non-null  object        
 3   Tipo Desp.      69333 non-null  object        
 4   CNPJ/CPF        69333 non-null  object        
 5   Fornecedor      69333 non-null  object        
 6   Documento       66370 non-null  object        
 7   Data            69333 non-null  datetime64[ns]
 8   Reembolso       69333 non-null  float64       
 9   Cod. Documento  69333 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 5.3+ MB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69333 entries, 0 to 69332
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Ano                      69333 non-null  int64         
 1   Mes                      69333 non-null  int64         
 2   Senador                  69333 non-null  object        
 3   Tipo Desp.               69333 non-null  object        
 4   CNPJ/CPF                 69333 non-null  object        
 5   Fornecedor               69333 non-null  object        
 6   Documento                66370 non-null  object        
 7   Data                     69333 non-null  datetime64[ns]
 8   Reembolso                69333 non-null  float64       
 9   Cod. Documento           69333 non-null  object        
 10  NomeCompletoParlamentar  69333 non-null  object        
 11  SexoParlamentar          69333 non-null  object        
 12  SiglaPartidoParlamentar  68210 n

None

In [201]:
df_completo['SiglaPartidoParlamentar'].fillna('S/Partido',inplace=True)
df_completo.rename(columns={
        "NomeCompletoParlamentar":"Nome Completo",
        "SexoParlamentar":"Sexo",
        "SiglaPartidoParlamentar":"Partido",
        "-UfParlamentar":"UF",
        "UrlFotoParlamentar":"Url da Foto Parlamentar"
    },inplace=True)
display(df_completo.info())
df_completo.to_csv('Dados CEASP Tratados.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69333 entries, 0 to 69332
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Ano                      69333 non-null  int64         
 1   Mes                      69333 non-null  int64         
 2   Senador                  69333 non-null  object        
 3   Tipo Desp.               69333 non-null  object        
 4   CNPJ/CPF                 69333 non-null  object        
 5   Fornecedor               69333 non-null  object        
 6   Documento                66370 non-null  object        
 7   Data                     69333 non-null  datetime64[ns]
 8   Reembolso                69333 non-null  float64       
 9   Cod. Documento           69333 non-null  object        
 10  Nome Completo            69333 non-null  object        
 11  Sexo                     69333 non-null  object        
 12  Partido                  69333 n

None